In [11]:
import pandas as pd
import os, json

In [12]:
ex = 23
ex_folder = f'experiments/ex{ex}'
if not os.path.exists(ex_folder):
    os.mkdir(ex_folder)

In [13]:
data = pd.read_csv('data/text.tsv', index_col=0, delimiter='\t')
labels = pd.read_csv('data/labels.tsv', index_col=0, delimiter='\t')
with open('data/value-categories.json', 'r') as file:
    vals = json.load(file) 

In [14]:
ls = []
fakeid = 0
for i, key in enumerate(labels.columns[:-1]):
    l = [0]*20
    l[i] = 1
    
    for j, key2 in enumerate(vals[key]):
        for elem in vals[key][key2]:
            ls.append([f'{key2} means {elem}', 'train', l.copy()])
            fakeid += 1
dfls = pd.DataFrame(ls, columns=['Text', 'Type', 'Labels'])

In [15]:
train_val_df = data.merge(labels, right_index=True, left_index=True).apply(
    lambda x: 
    pd.Series((
        f'{x.Conclusion} {x.Stance} {x.Premise}', # ex0, ex3, ex4, ex5, ex6, ex8, ex9, ex10, ex11, ex12, ex13, ex14, ex15, ex16, ex17, ex22, ex23
        #f'{x.Conclusion} {"[A]" if x.Stance == "against" else "[F]"} {x.Premise}', # ex1
        #f'[C] {x.Conclusion} [/C] {"[A]" if x.Stance == "against" else "[F]"} [P] {x.Premise} [/P]', ex2
        #f'{x.Premise} {"therefore" if "favor" in x.Stance else "nevertheless"} {x.Conclusion}', # ex7, ex21
        
        x.Type_x, # ex0, ex1, ex2, ex3, ex6
        #x.Type_x if 'favor' in x.Stance else 'REM', # ex4,
        #x.Type_x if 'against' in x.Stance else 'REM', # ex5,
        
        list(labels.loc[x.name].values[:-1])
    )), 
    axis=1
)
train_val_df.columns = ['Text', 'Type', 'Labels']
train_val_df = train_val_df.loc[train_val_df['Type']!='REM']
special_tokens = []
#special_tokens = ['[A]', '[F]']
#special_tokens = ['[C]', '[/C]', '[A]', '[F]', '[P]', '[/P]']
#train_val_df = pd.concat([train_val_df, dfls]) # ex8
#train_val_df = pd.concat([train_val_df, dfls, dfls]) # ex9
#train_val_df = pd.concat([train_val_df, dfls, dfls, dfls, dfls, dfls, dfls]).sample(frac=1) # ex10, ex12, ex14, ex16, ex22
#train_val_df = pd.concat([train_val_df, dfls, dfls, dfls, dfls]).sample(frac=1) # ex17, ex18, ex19, ex20
train_val_df.index.name = 'Argument ID'

In [16]:
test_df = data.apply(
    lambda x: 
    pd.Series((
        f'{x.Conclusion} {x.Stance} {x.Premise}', # ex0, ex3, ex4, ex5, ex6, ex8, ex9, ex10, ex11, ex12, ex13, ex14, ex15, ex16, ex17, ex22, ex23
        #f'{x.Conclusion} {"[A]" if x.Stance == "against" else "[F]"} {x.Premise}', # ex1
        #f'[C] {x.Conclusion} [/C] {"[A]" if x.Stance == "against" else "[F]"} [P] {x.Premise} [/P]', ex2
        #f'{x.Premise} {"therefore" if "favor" in x.Stance else "nevertheless"} {x.Conclusion}', # ex7, ex21
        
        x.Type, # ex0, ex1, ex2, ex3, ex6
        #x.Type_x if 'favor' in x.Stance else 'REM', # ex4,
        #x.Type_x if 'against' in x.Stance else 'REM', # ex5,
    )), 
    axis=1
)
test_df.columns = ['Text', 'Type']
test_df.index.name = 'Argument ID'

In [17]:
train_val_df.loc[train_val_df.Type=='train'].drop('Type', axis=1).to_csv(f'{ex_folder}/train-mlabel.tsv', sep='\t')

train_val_df.loc[train_val_df.Type=='val'].drop('Type', axis=1).to_csv(f'{ex_folder}/val-mlabel.tsv', sep='\t')
train_val_df.loc[train_val_df.Type=='zhihu'].drop('Type', axis=1).to_csv(f'{ex_folder}/zhihu-mlabel.tsv', sep='\t')

test_df.loc[test_df.Type=='test'].drop('Type', axis=1).to_csv(f'{ex_folder}/test-mlabel.tsv', sep='\t')
test_df.loc[test_df.Type=='nahj'].drop('Type', axis=1).to_csv(f'{ex_folder}/nahj-mlabel.tsv', sep='\t')
test_df.loc[test_df.Type=='nyt'].drop('Type', axis=1).to_csv(f'{ex_folder}/nyt-mlabel.tsv', sep='\t')

pd.DataFrame(special_tokens).to_csv(f'{ex_folder}/custom-tokens-mlabel.csv')

In [18]:
train_val_df

,Text,Type,Labels
Argument ID,,,
A01002,We should ban human cloning in favor of we sho...,train,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
A01005,We should ban fast food in favor of fast food ...,train,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
A01006,We should end the use of economic sanctions ag...,train,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
A01007,We should abolish capital punishment against c...,train,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, ..."
A01008,We should ban factory farming against factory ...,train,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, ..."
...,...,...,...
C26096,Carbon emissions trading should be limited. in...,zhihu,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
C26097,Carbon emissions trading should be limited. in...,zhihu,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
C26098,Carbon emissions trading should be limited. ag...,zhihu,"[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [19]:
a = [len(row.tolist()[0]) for i, row in test_df.iterrows()]

In [20]:
max(a)

880